In [1]:
# Imports
import configparser
from pathlib import Path
import pathlib
from direct_damages import damagescanner_rail_track as ds
import pandas as pd
import geopandas as gpd
# import datetime
from ci_adapt_utilities import *
import pickle
# import networkx as nx
import shapely

In [2]:
# Load configuration with ini file (created running config.py)
config_file=r'C:\repos\ci_adapt\config_ci_adapt.ini'
config = configparser.ConfigParser()
config.read(config_file)

p = Path('..')
hazard_type = config.get('DEFAULT', 'hazard_type')
infra_type = config.get('DEFAULT', 'infra_type')
country_code = config.get('DEFAULT', 'country_code')
country_name = config.get('DEFAULT', 'country_name')
hazard_data_subfolders = config.get('DEFAULT', 'hazard_data_subfolders')
asset_data = config.get('DEFAULT', 'asset_data')
vulnerability_data = config.get('DEFAULT', 'vulnerability_data')
data_path = Path(pathlib.Path.home().parts[0]) / 'Data'
interim_data_path = data_path / 'interim' / 'collected_flood_runs'

In [3]:
# Define costs for different transport modes
average_train_load_tons = (896+1344+2160+1344+896+896+1344+1512+896+390)/10 # in Tons per train. Source: Kennisinstituut voor Mobiliteitsbeleid. 2023. Cost Figures for Freight Transport – final report
average_train_cost_per_ton_km = (0.014+0.018+0.047+0.045)/4 # in Euros per ton per km. Source: Kennisinstituut voor Mobiliteitsbeleid. 2023. Cost Figures for Freight Transport – final report
average_road_cost_per_ton_km = (0.395+0.375+0.246+0.203+0.138+0.153+0.125+0.103+0.122+0.099)/10 # in Euros per ton per km. Source: Kennisinstituut voor Mobiliteitsbeleid. 2023. Cost Figures for Freight Transport – final report

In [4]:
# Define basic functions

# Define other functions (development)
def find_basin_lists(basins, regions):
    intersect_basins_regions = gpd.overlay(basins, regions, how='intersection')
    exclude_main_rivers=intersect_basins_regions.loc[intersect_basins_regions['ORDER']==1]

    basins_exclusion_list = [x for x in exclude_main_rivers['HYBAS_ID'].values]
    basin_list_tributaries = set([x for x in intersect_basins_regions['HYBAS_ID'].values if x not in basins_exclusion_list]) 
    basin_list_full_flood = set(intersect_basins_regions['HYBAS_ID'].values)
    return basin_list_tributaries, basin_list_full_flood



In [5]:
# Read exposure data (OSM, OpenStreetMap contributors (2024) / osm-flex)
assets_path = data_path / asset_data
assets=preprocess_assets(assets_path)

# Add buffer to assets to do area intersect and create dictionaries for quicker lookup
buffered_assets = ds.buffer_assets(assets)
geom_dict = assets['geometry'].to_dict()
type_dict = assets['asset'].to_dict()

print(f"{len(assets)} assets loaded.")

# Read vulnerability and maximum damage data from Nirandjan, S., et al. (2024)
curve_types = {'rail': ['F8.1']}
infra_curves, maxdams = ds.read_vul_maxdam(data_path, hazard_type, infra_type)
max_damage_tables = pd.read_excel(data_path / vulnerability_data / 'Table_D3_Costs_V1.0.0.xlsx',sheet_name='Cost_Database',index_col=[0])
print(f'Found matching infrastructure curves for: {infra_type}')

122720 assets loaded.
Found matching infrastructure curves for: rail


In [6]:
#load pickled shortest paths, disrupted edges, shortest paths, graph
shortest_paths = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'shortest_paths.pkl', 'rb'))
disrupted_edges_by_basin = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'disrupted_edges_by_basin.pkl', 'rb'))
graph_r0 = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'graph_0.pkl', 'rb'))
disrupted_shortest_paths = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'disrupted_shortest_paths.pkl', 'rb'))
event_impacts = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'event_impacts.pkl', 'rb'))
print('Loaded data from baseline impact assessment')

if 'collect_output' not in locals():
    collect_output_path = f'C:/Data/interim/collected_flood_runs/sample_collected_run.pkl'
    with open(collect_output_path, 'rb') as f:
        collect_output = pickle.load(f)

Loaded data from baseline impact assessment


In [7]:
graph_v0=create_virtual_graph(graph_r0)
graph_v=graph_v0.copy()

Max weight: 34076035
Max capacity: 1
Success: only int type values


In [8]:
# Set adaptation data
l1_l2_adapt_path=None#Path(r'C:\Data\input\adaptations\l2_tributary.geojson')
# adapted_area=create_adaptations_gdf(adapt_path)
added_links =[(4424116, 219651487), (219651487, 111997047)]
l4_adapt_path=None#Path(r'C:\Data\input\adaptations\l4_tributary.geojson')


if l1_l2_adapt_path is not None:
    adapted_area = gpd.read_file(l1_l2_adapt_path).to_crs(3857)
else:
    adapted_area = None
adaptations_df=create_adaptation_df(adapted_area)
if l4_adapt_path is not None:
    adapted_route_area = gpd.read_file(l4_adapt_path).to_crs(3857)
else:
    adapted_route_area = None

In [9]:
adapted_area

In [10]:
asset_options={'bridge_design_rp':'M',
               'tunnel_design_rp':'M'}
return_period_dict = {'_H_': 10,'_M_': 100,'_L_': 200}
adaptation_unit_costs = {'fwall': 7000.97, #considering floodwall in Germany
                         'viaduct': 22500.00*1.3, #considering viaduct is 30% more expensive than ground level
                         'bridge': 3700*10}  #considering bridge of 10m deck width
rp_spec_priority = set_rp_priorities(return_period_dict)
rp_spec_priority

('L', 'M', 'H', None)

Apply adaptations

In [11]:
if adapted_area is not None:
    assets_to_adapt = filter_assets_to_adapt(assets, adapted_area)
    adapted_assets = assets.loc[assets.index.isin(assets_to_adapt.index)].copy()
    adapted_assets = add_adaptation_columns(adapted_assets)

    for (adaptation_id, ad) in adapted_area.iterrows():
        affected_assets=assets_to_adapt.loc[assets_to_adapt['adaptation_id']==adaptation_id].copy()  
        rp_specs = set(affected_assets['rp_spec'])

        for hazard_map in collect_output.keys():
            haz_rp=hazard_map.split('_')[-3]
            print(haz_rp)
            if haz_rp not in rp_specs:
                print('cont 0')
                continue
            overlay_assets = load_baseline_run(hazard_map, interim_data_path, only_overlay=True)
            if set(overlay_assets.asset.values).isdisjoint(affected_assets.index):
                print('cont 1')
                continue
            else: 
                print('else 1')
                overlay_assets, hazard_numpified_list = load_baseline_run(hazard_map, interim_data_path)
                adapted_assets = apply_asset_adaptations_in_haz_area(adapted_assets, affected_assets, overlay_assets, hazard_numpified_list, rp_spec_priority)
else:
    adapted_assets = assets.iloc[0:0].copy()
    adapted_assets = add_adaptation_columns(adapted_assets)
l3_adaptation_costs = {}
for i,osm_id_pair in enumerate(added_links):
    graph_v, l3_ad_cost = add_l3_adaptation(graph_v, osm_id_pair, adaptation_unit_cost=adaptation_unit_costs['bridge'])
    l3_adaptation_costs[osm_id_pair] = l3_ad_cost
    l3_ad_sum=[i, 'NA', 3, 'NA', l3_ad_cost]
    adaptations_df.loc[i] = l3_ad_sum

if adapted_route_area is not None:
    demand_reduction_dict = add_l4_adaptation(graph_v, shortest_paths, adapted_route_area, demand_reduction=1.0)
    for i,(o,d) in enumerate(demand_reduction_dict.keys()):
        # print('i: ', i)
        # print('od: ',(o,d))
        l4_ad_sum=[i, 'NA', 4, ((o,d),demand_reduction_dict[(o,d)]),0]
        adaptations_df.loc[i] = l4_ad_sum
else:
    demand_reduction_dict = {}

Applying adaptation: new connection between assets with osm_id  (4424116, 219651487)
Level 3 adaptation
Applying adaptation: new connection between assets with osm_id  (219651487, 111997047)
Level 3 adaptation


In [12]:
adapted_assets.head(3)

,osm_id,asset,name,gauge,electrified,voltage,bridge,maxspeed,service,tunnel,...,buffered,l1_adaptation,l1_rp_spec,l2_adaptation_exp,l2_adaptation_vul,l2_rp_spec,l3_adaptation,l3_rp_spec,l4_adaptation,l4_rp_spec


In [13]:
direct_damages_adapted = {}
indirect_damages_adapted = {}

In [14]:
local_haz_path=data_path/r'Floods\Germany\fluvial_undefended\raw_subsample\validated_geometries'
hazard_data_list = ds.read_hazard_data(local_haz_path, hazard_type, country=country_name, subfolders=None)
print(f'Found {len(hazard_data_list)} hazard maps.')
if adapted_area is not None:
    adapted_area=adapted_area.explode().reset_index(drop=True)
    adapted_area['geometry'] = adapted_area['geometry'].apply(
        lambda x: shapely.LineString(x.exterior) if isinstance(x, (shapely.Polygon, shapely.MultiPolygon)) else x
    )
    adapted_area['buffered'] = shapely.buffer(adapted_area.geometry.values,distance=1)

    l1_adaptation_costs = {}
    for (adaptation_id, ad) in adapted_area.iterrows():
        if ad.adapt_level != 1:
            continue

        for i, single_footprint in enumerate(hazard_data_list):
            hazard_map = single_footprint.parts[-1].split('.')[0]     
            
            haz_rp=hazard_map.split('_')[-3]
            if haz_rp != ad.rp_spec.upper():
                continue
            
            overlay_assets = load_baseline_run(hazard_map, interim_data_path, only_overlay=True)
            if set(overlay_assets.asset.values).isdisjoint(assets_to_adapt.index):
                print('No assets to adapt')
                continue
            if 'fwall' not in ad.prot_area:
                continue
            try:
                adaptation_gdf=gpd.GeoDataFrame(adapted_area.iloc[[adaptation_id]])
                l1_adaptation_costs[adaptation_id] = process_adap_dat(single_footprint, adaptation_gdf, hazard_numpified_list, adaptation_unit_cost=adaptation_unit_costs['fwall'])
                continue

            except Exception as e:
                print(f'Error occurred in {hazard_map}: {str(e)}')
                continue

## MERGE THIS AND THE FOLLOWING CELL AND CALL THEM AS FUNCTIONS

Found 183 hazard maps.


In [15]:
l2_adaptation_costs = {}
overlay_assets_lists = {'flood_DERP_RW_M':[], 'flood_DERP_RW_H':[], 'flood_DERP_RW_L':[]}
# overlay_assets_full = pd.DataFrame()
adaptation_run_full = {'flood_DERP_RW_M':[{},{}], 'flood_DERP_RW_H':[{},{}], 'flood_DERP_RW_L':[{},{}]}
for hazard_map in collect_output.keys():
    hm_full_id=hazard_map.split('_4326')[0]

    print('\nProcessing hazard map: ', hazard_map)
    map_rp_spec = hazard_map.split('_')[-3]
    overlay_assets, hazard_numpified_list = load_baseline_run(hazard_map, interim_data_path)
    overlay_assets_lists[hm_full_id].extend(overlay_assets.asset.values.tolist())
    # overlay_assets_full = pd.concat([overlay_assets_full, overlay_assets])
    adaptation_run = run_direct_damage_reduction_by_hazmap(assets, geom_dict, overlay_assets, hazard_numpified_list, collect_output[hazard_map], adapted_assets, map_rp_spec=map_rp_spec, rp_spec_priority=rp_spec_priority, reporting=False, adaptation_unit_cost=adaptation_unit_costs['viaduct'])

    # Sum the values for the first dictionary in the tuple
    adaptation_run_full[hm_full_id][0] = {k: [adaptation_run_full[hm_full_id][0].get(k, [0, 0])[0] + adaptation_run[0].get(k, [0, 0])[0], 
                                adaptation_run_full[hm_full_id][0].get(k, [0, 0])[1] + adaptation_run[0].get(k, [0, 0])[1]] 
                            for k in set(adaptation_run_full[hm_full_id][0]) | set(adaptation_run[0])}

    # Sum the values for the second dictionary in the tuple
    adaptation_run_full[hm_full_id][1] = {k: [adaptation_run_full[hm_full_id][1].get(k, [0, 0])[0] + adaptation_run[1].get(k, [0, 0])[0], 
                                adaptation_run_full[hm_full_id][1].get(k, [0, 0])[1] + adaptation_run[1].get(k, [0, 0])[1]] 
                            for k in set(adaptation_run_full[hm_full_id][1]) | set(adaptation_run[1])}

    direct_damages_adapted[hazard_map]=adaptation_run[1]
    l2_adaptation_costs[hazard_map] = adaptation_run[2]
    if direct_damages_adapted[hazard_map]=={} and 3 not in adaptations_df.adapt_level.unique() and 4 not in adaptations_df.adapt_level.unique():
        indirect_damages_adapted[hazard_map]={}
        continue
    disrupted_edges = disrupted_edges_by_basin[hazard_map] if hazard_map in disrupted_edges_by_basin.keys() else []
    indirect_damages_adapted[hazard_map] = run_indirect_damages_by_hazmap(adaptation_run, assets, hazard_map, overlay_assets, disrupted_edges, shortest_paths, graph_v, average_train_load_tons, average_train_cost_per_ton_km, average_road_cost_per_ton_km, demand_reduction_dict)


Processing hazard map:  flood_DERP_RW_H_4326_2080410170
Found matching infrastructure curves for: rail
-- Calculating direct damages --
2024-10-08 12:57:06 - Calculating adapted damages for assets...


100%|██████████| 30/30 [00:00<00:00, 15015.41it/s]

30 assets with no change.


-- Calculating indirect damages --
disrupted_edges baseline:  [('node_11754', 'node_12067'), ('node_12067', 'node_11754'), ('node_63681', 'node_63707'), ('node_63707', 'node_63681'), ('node_92367', 'node_94053'), ('node_94053', 'node_92367')]
disrupted_edges_adapted:  [('node_11754', 'node_12067'), ('node_12067', 'node_11754'), ('node_63681', 'node_63707'), ('node_63707', 'node_63681'), ('node_92367', 'node_94053'), ('node_94053', 'node_92367')]
flood_DERP_RW_H_4326_2080410170 467036.2736560443

Processing hazard map:  flood_DERP_RW_H_4326_2080410430
Found matching infrastructure curves for: rail
-- Calculating direct damages --
2024-10-08 12:57:19 - Calculating adapted damages for assets...


100%|██████████| 13/13 [00:00<00:00, 4339.86it/s]

...

6 assets with no change.


-- Calculating indirect damages --
disrupted_edges baseline:  [('node_80804', 'node_129872'), ('node_80835', 'node_80836'), ('node_80836', 'node_80835'), ('node_80836', 'node_107318'), ('node_80836', 'node_107322'), ('node_107318', 'node_80836'), ('node_107322', 'node_80836'), ('node_107322', 'node_129871'), ('node_107322', 'node_129872'), ('node_129871', 'node_107322'), ('node_129872', 'node_107322'), ('node_129872', 'node_80804')]
disrupted_edges_adapted:  [('node_80804', 'node_129872'), ('node_80835', 'node_80836'), ('node_80836', 'node_80835'), ('node_80836', 'node_107318'), ('node_80836', 'node_107322'), ('node_107318', 'node_80836'), ('node_107322', 'node_80836'), ('node_107322', 'node_129871'), ('node_107322', 'node_129872'), ('node_129871', 'node_107322'), ('node_129872', 'node_107322'), ('node_129872', 'node_80804')]
flood_DERP_RW_M_4326_2080448120 18567.89963685935

Processing hazard map:  flood_DERP_RW_M_4326_2080451930
Found matching infrastructure curves for: rail
-- Calcu

100%|██████████| 7/7 [00:00<00:00, 7015.56it/s]

7 assets with no change.


-- Calculating indirect damages --
disrupted_edges baseline:  [('node_16111', 'node_16112'), ('node_16112', 'node_16111'), ('node_80835', 'node_80836'), ('node_80836', 'node_80835'), ('node_107322', 'node_129871'), ('node_129871', 'node_107322')]
disrupted_edges_adapted:  [('node_16111', 'node_16112'), ('node_16112', 'node_16111'), ('node_80835', 'node_80836'), ('node_80836', 'node_80835'), ('node_107322', 'node_129871'), ('node_129871', 'node_107322')]
flood_DERP_RW_M_4326_2080451930 18567.89963685935


In [16]:
oad=overlay_assets.to_dict()

In [17]:
full_flood_event=pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'full_flood_event.pkl', 'rb'))
all_disrupted_edges = pickle.load(open(data_path / 'interim' / 'indirect_damages' / 'all_disrupted_edges.pkl', 'rb'))

In [18]:
indirect_damages_adapted_full={}
for hazard_map in full_flood_event.keys():
    print('\nProcessing hazard map: ', hazard_map)
    overlay_assets_full_dict = {i:overlay_assets_lists[hazard_map][i] for i in range(len(overlay_assets_lists[hazard_map]))}
    overlay_assets_full=pd.DataFrame(overlay_assets_full_dict, index=['asset']).T
    map_rp_spec = hazard_map.split('_')[-3]
    # overlay_assets, hazard_numpified_list = load_baseline_run(hazard_map, interim_data_path)
    # adaptation_run = run_direct_damage_reduction_by_hazmap(assets, geom_dict, overlay_assets, hazard_numpified_list, full_flood_event[hazard_map], adapted_assets, map_rp_spec=map_rp_spec, rp_spec_priority=rp_spec_priority, reporting=False, adaptation_unit_cost=adaptation_unit_costs['viaduct'])
    # direct_damages_adapted[hazard_map]=adaptation_run[1]
    # l2_adaptation_costs[hazard_map] = adaptation_run[2]
    # if direct_damages_adapted[hazard_map]=={} and 3 not in adaptations_df.adapt_level.unique() and 4 not in adaptations_df.adapt_level.unique():
    #     indirect_damages_adapted[hazard_map]={}
    #     continue
    # disrupted_edges = disrupted_edges_by_basin[hazard_map] if hazard_map in disrupted_edges_by_basin.keys() else []
    indirect_damages_adapted_full[hazard_map] = run_indirect_damages_by_hazmap(adaptation_run_full[hazard_map], assets, hazard_map, overlay_assets_full, all_disrupted_edges[hazard_map], shortest_paths, graph_v, average_train_load_tons, average_train_cost_per_ton_km, average_road_cost_per_ton_km, demand_reduction_dict)


Processing hazard map:  flood_DERP_RW_H
-- Calculating indirect damages --
disrupted_edges baseline:  [('node_100793', 'node_118139'), ('node_127017', 'node_127013'), ('node_14952', 'node_14956'), ('node_59250', 'node_96304'), ('node_20902', 'node_20916'), ('node_12067', 'node_11754'), ('node_60291', 'node_60296'), ('node_33169', 'node_102934'), ('node_135304', 'node_29410'), ('node_121578', 'node_121576'), ('node_105428', 'node_102238'), ('node_103785', 'node_103786'), ('node_126745', 'node_12116'), ('node_132930', 'node_19620'), ('node_121571', 'node_121575'), ('node_100796', 'node_100800'), ('node_129946', 'node_129943'), ('node_101945', 'node_101946'), ('node_91720', 'node_118130'), ('node_61913', 'node_61912'), ('node_131331', 'node_67509'), ('node_102934', 'node_33169'), ('node_63853', 'node_63848'), ('node_15566', 'node_15565'), ('node_29570', 'node_29571'), ('node_10523', 'node_15565'), ('node_29571', 'node_29570'), ('node_121577', 'node_121581'), ('node_61759', 'node_61762'),

In [19]:
for hazard_map in collect_output.keys():
    if direct_damages_adapted[hazard_map]=={}:
       direct_damages_adapted[hazard_map]=collect_output[hazard_map]
    if indirect_damages_adapted[hazard_map]=={}:
       indirect_damages_adapted[hazard_map]=event_impacts[hazard_map] if hazard_map in event_impacts.keys() else 0.0

In [20]:
# Save results
adapt_id='_l3_trib'
direct_damages_adapted_path = data_path / 'output' / f'adapted_direct_damages{adapt_id}.pkl'
indirect_damages_adapted_path = data_path / 'output' / f'adapted_indirect_damages{adapt_id}.pkl'
indirect_damages_adapted_full_path = data_path / 'output' / f'adapted_indirect_damages_full{adapt_id}.pkl'
adaptations_df_path = data_path / 'output' / f'adaptations{adapt_id}.csv'

with open(direct_damages_adapted_path, 'wb') as f:
    pickle.dump(direct_damages_adapted, f)
with open(indirect_damages_adapted_path, 'wb') as f:
    pickle.dump(indirect_damages_adapted, f)
with open(indirect_damages_adapted_full_path, 'wb') as f:    
    pickle.dump(indirect_damages_adapted_full, f)

adaptations_df.to_csv(adaptations_df_path, index=False)


In [21]:
adapted_assets_path = data_path / 'output' / f'adapted_assets{adapt_id}.pkl'
with open(adapted_assets_path, 'wb') as f:
    pickle.dump(adapted_assets, f)

In [22]:
direct_damages_baseline_sum = {
    key: (
        sum(v[0] for v in collect_output[key].values()), 
        sum(v[1] for v in collect_output[key].values())
        ) 
        for key in collect_output if key in collect_output.keys()
        }
direct_damages_adapted_sum = {
    key: (
        sum(v[0] for v in direct_damages_adapted[key].values()), 
        sum(v[1] for v in direct_damages_adapted[key].values())
        ) 
        for key in direct_damages_adapted if key in direct_damages_adapted.keys()
        }

direct_damages_diff = {
    key: (
        direct_damages_baseline_sum[key][0] - direct_damages_adapted_sum[key][0], 
        direct_damages_baseline_sum[key][1] - direct_damages_adapted_sum[key][1]
        ) for key in direct_damages_baseline_sum
        }

direct_damages_diff


{'flood_DERP_RW_H_4326_2080410170': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080410430': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080410540': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080410660': (0, 0),
 'flood_DERP_RW_H_4326_2080410760': (0, 0),
 'flood_DERP_RW_H_4326_2080411370': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080416200': (0, 0),
 'flood_DERP_RW_H_4326_2080416210': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080418600': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080418720': (0, 0),
 'flood_DERP_RW_H_4326_2080418880': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080418890': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080419070': (0, 0),
 'flood_DERP_RW_H_4326_2080419180': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080420240': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080420340': (0, 0),
 'flood_DERP_RW_H_4326_2080421680': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080421770': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080428080': (0, 0),
 'flood_DERP_RW_H_4326_2080428160': (0.0, 0.0),
 'flood_DERP_RW_H_4326_2080428500': (0, 0),
 'flood_DERP_RW_H_4326_2

Visualisations

In [18]:
pd.options.display.float_format = "{:,.2f}".format

total_damages_adapted={}
for hazard_map in direct_damages_adapted.keys():

    map_rp_spec = hazard_map.split('_')[-3]

    adap_costs=adaptations_df['adaptation_cost']
    summed_adaptation_costs = sum(adap_costs)

    #direct damages
    dd_bl=direct_damages_adapted[hazard_map][0]
    summed_dd_bl_lower=sum([v[0] for v in dd_bl.values()])
    summed_dd_bl_upper=sum([v[1] for v in dd_bl.values()])
    dd_ad=direct_damages_adapted[hazard_map][1]
    summed_dd_ad_lower=sum([v[0] for v in dd_ad.values()])
    summed_dd_ad_upper=sum([v[1] for v in dd_ad.values()])
    
    #indirect damages
    if hazard_map not in event_impacts.keys():
        print(f'{hazard_map} not in event_impacts')
        id_bl=0
        id_ad=0
        id_ad_cleaned=0
    else:
        id_bl=event_impacts[hazard_map]
        id_ad=indirect_damages_adapted[hazard_map]
        id_ad_cleaned = 0 if id_ad == 99999999999999 else id_ad

    total_damages_adapted[hazard_map]=(map_rp_spec, summed_adaptation_costs, (summed_dd_bl_lower, summed_dd_bl_upper), (summed_dd_ad_lower, summed_dd_ad_upper), id_bl, id_ad_cleaned)
    
total_damages_adapted_df=pd.DataFrame(total_damages_adapted)
total_damages_adapted_df=total_damages_adapted_df.T
total_damages_adapted_df.columns=['return_period','summed_adaptation_costs', 'summed_dd_bl', 'summed_dd_ad', 'indirect damage baseline [€]', 'indirect damage adapted [€]']

# round and turn to million euros for reporting
total_damages_adapted_df_mill=total_damages_adapted_df.copy()
total_damages_adapted_df_mill['summed_adaptation_costs [M€]']=total_damages_adapted_df_mill['summed_adaptation_costs']/1e6
total_damages_adapted_df_mill['summed_dd_bl [M€]']=total_damages_adapted_df_mill['summed_dd_bl'].apply(lambda x: (x[0]/1e6, x[1]/1e6))
total_damages_adapted_df_mill['summed_dd_ad [M€]']=total_damages_adapted_df_mill['summed_dd_ad'].apply(lambda x: (x[0]/1e6, x[1]/1e6))
total_damages_adapted_df_mill['indirect damage baseline [M€]']=total_damages_adapted_df_mill['indirect damage baseline [€]']/1e6
total_damages_adapted_df_mill['indirect damage adapted [M€]']=total_damages_adapted_df_mill['indirect damage adapted [€]']/1e6
total_damages_adapted_df_mill.drop(['summed_adaptation_costs','summed_dd_bl', 'summed_dd_ad', 'indirect damage baseline [€]', 'indirect damage adapted [€]'], axis=1, inplace=True)



custom_order = ['H', 'M', 'L', 'Unknown']

total_damages_adapted_df_mill['return_period'] = pd.Categorical(total_damages_adapted_df_mill['return_period'], 
                                                                categories=custom_order, ordered=True)
sorted_total_damages_adapted_df_mill = total_damages_adapted_df_mill.sort_values(by='return_period', ascending=True)
sorted_total_damages_adapted_df_mill

KeyError: 0

In [19]:
shortest_paths_assets={}
od_assets=[]
o_geoms=[]
d_geoms=[]
for (o,d), (path, demand) in shortest_paths.items():
    od_assets_by_sp=[]
    o_geoms.append(graph_v.nodes[o]['geometry'])
    d_geoms.append(graph_v.nodes[d]['geometry'])

    for i in range(len(path)-1):
        x=graph_v.edges[path[i], path[i+1], 0]
        od_assets_by_sp.append(x['osm_id'])
        od_assets.append(x['osm_id'])

    shortest_paths_assets[(o,d)]=(od_assets_by_sp, demand)
assets_sps=assets.loc[assets['osm_id'].isin(set(od_assets))].copy()


# Repeat for shortest paths under adapted conditions
shortest_paths_adapted_assets={}
for flood_map, od_dict in disrupted_shortest_paths.items():
    for (o,d), (path, demand) in od_dict.items():
        od_assets_by_sp_adapted=[]
        for i in range(len(path)-1):
            x=graph_v.edges[path[i], path[i+1], 0]
            od_assets_by_sp_adapted.append(x['osm_id'])
        shortest_paths_adapted_assets[flood_map, (o,d)]=(od_assets_by_sp_adapted, demand)


df_shortest_paths=pd.DataFrame(shortest_paths_assets).T
df_shortest_paths.columns=['path', 'demand']
df_shortest_paths


KeyError: 'osm_id'

In [20]:

df_shortest_paths_adapted=pd.DataFrame(shortest_paths_adapted_assets).T
df_shortest_paths_adapted.columns=['path', 'demand']
df_shortest_paths_adapted


path  \
flood_DERP_RW_H_4326_2080410170 (node_13219, node_63664)   [27333502, 159679113, 159679100, 827463941, 31...   
                                (node_63664, node_13219)   [816162304, 114655435, 813843121, 114655444, 1...   
flood_DERP_RW_H_4326_2080410430 (node_13219, node_103249)  [816162312, 694690444, 816162311, 816162311, 1...   
                                (node_103249, node_13219)  [235878134, 235878124, 235878120, 481305333, 4...   
flood_DERP_RW_H_4326_2080410540 (node_13219, node_103249)  [816162312, 694690444, 816162311, 114681879, 1...   
                                (node_103249, node_13219)  [235878134, 235878124, 235878120, 481305333, 4...   
                                (node_63800, node_13219)   [114694798, 114695476, 114695477, 114704935, 3...   
                                (node_13219, node_63800)   [816162312, 694690444, 816162311, 816162311, 1...   
flood_DERP_RW_H_4326_2080411370 (node_110851, node_59095)  [303605744, 242732900, 601085138, 111584614, 6...   
                                (node_59095, node_110851)  [110888542, 110888545, 110888548, 110888522, 1...   

                                                          demand  
flood_DERP_RW_H_4326_2080410170 (node_13219, node_63664)      58  
                                (node_63664, node_13219)      59  
flood_DERP_RW_H_4326_2080410430 (node_13219, node_103249)     24  
                                (node_103249, node_13219)     27  
flood_DERP_RW_H_4326_2080410540 (node_13219, node_103249)     24  
                                (node_103249, node_13219)     27  
                                (node_63800, node_13219)      44  
                                (node_13219, node_63800)      46  
flood_DERP_RW_H_4326_2080411370 (node_110851, node_59095)    378  
                                (node_59095, node_110851)    392

In [24]:

l3_geometries = {}
for u, v, k, attr in graph_v.edges(keys=True, data=True):
    if 'osm_id' not in attr:
        continue
    if 'l3_adaptation' in attr['osm_id']:
        # Ensure the geometry is a valid Shapely geometry object
        geometry = attr['geometry']
        if isinstance(geometry, list) and len(geometry) == 1 and isinstance(geometry[0], shapely.LineString):
            geometry = geometry[0]
        if isinstance(geometry, shapely.LineString):
            l3_geometries[(u, v)] = geometry
        else:
            print(f"Invalid geometry for edge ({u}, {v}): {geometry}")

gdf_l3_edges = gpd.GeoDataFrame.from_dict(l3_geometries, orient='index', columns=['geometry'], geometry='geometry', crs=3857)
gdf_l3_edges.reset_index(inplace=True)

gdf_l3_edges = gpd.GeoDataFrame(list(l3_geometries.items()), columns=['edge', 'geometry'], geometry='geometry', crs=3857)

adapted_assets_path = data_path / 'output' / f'adapted_assets{adapt_id}.pkl'
with open(adapted_assets_path, 'wb') as f:
    pickle.dump(gdf_l3_edges, f)

In [22]:
# EXPAND FOR VISUALISATION SCRIPT
# Create visualisation for the basin and the discharge points
from lonboard import Map, PolygonLayer, PathLayer, BaseLayer, ScatterplotLayer
import ast
# MIRACA color scheme
color_string = config.get('DEFAULT', 'miraca_colors')
miraca_colors = ast.literal_eval(color_string)

basin_id = 2080430320
rp_vis = 'M'
overlay_assets = load_baseline_run(f'flood_DERP_RW_{rp_vis}_4326_{basin_id}', interim_data_path, only_overlay=True)
    
# Set path for basin to add to visualization
basin_path = rf'C:\Data\Floods\basins\hybas_eu_{int(basin_id)}.shp'

# Generate the basin layer
basin = gpd.read_file(basin_path)
layer_basin = PolygonLayer.from_geopandas(basin,
    get_fill_color=miraca_colors['grey_100'],
    get_line_color=miraca_colors['accent green'], get_line_width=10,
    auto_highlight=False,
    filled=True, opacity=0.1)

# Generate od layer for visualization
od_geoms=o_geoms+d_geoms
od_geoms_gdf=gpd.GeoDataFrame(geometry=od_geoms).set_crs(3857)
layer_od = ScatterplotLayer.from_geopandas(od_geoms_gdf, get_fill_color=miraca_colors['red_danger'], get_radius=200, opacity=0.8, auto_highlight=True)



c:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\Lib\site-packages\lonboard\_geoarrow\ops\reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


In [23]:

# Set path for the protected area to add to visualization
adapt_path = l1_l2_adapt_path
if adapt_path is not None:
    adapt_area = gpd.read_file(adapt_path)
    if len(adapt_area.adapt_level.unique())==1 and adapt_area.adapt_level.unique()[0]==1:
        layer_adapted_area = PolygonLayer.from_geopandas(adapt_area,
                                                         get_fill_color=miraca_colors['green_800'],
                                                         get_line_color=miraca_colors['primary blue'], get_line_width=10,
                                                         auto_highlight=False, filled=True, opacity=0.1)
    else:
        layer_adapted_area = None
else:
    layer_adapted_area = None

# Create layer for assets for visualization
layer_assets = PathLayer.from_geopandas(assets.drop(columns=['buffered', 'other_tags']), get_width=80, get_color=miraca_colors['grey_400'], auto_highlight=True, )
layer_shortest_path_assets = PathLayer.from_geopandas(assets_sps.drop(columns=['buffered', 'other_tags']), get_width=80, get_color=miraca_colors['black'], auto_highlight=True)
affected_assets = [asset_id for asset_id in list(set(overlay_assets.asset.values))]
layer_affected_assets = PathLayer.from_geopandas(assets.iloc[affected_assets].drop(columns=['buffered', 'other_tags']), get_width=80, get_color=miraca_colors['red_danger'], auto_highlight=True)

try:
    layer_protected_assets = PathLayer.from_geopandas(adapted_assets.drop(columns=['buffered', 'other_tags']), get_width=90, get_color=miraca_colors['green_success'], auto_highlight=True)
except:
    layer_protected_assets = None
try:
    layer_l3_edges = PathLayer.from_geopandas(gdf_l3_edges, get_width=90, get_color=miraca_colors['green_success'])
except:
    layer_l3_edges = None
layer_assets_raw = [layer_assets, layer_shortest_path_assets, layer_affected_assets, layer_protected_assets, layer_l3_edges]
layers_assets = [layer for layer in layer_assets_raw if layer is not None]

# Generate flood layers and protection layers for visualization
flood_plot_path=rf'Floods\Germany\basin_intersections\DERP_RW_{rp_vis}_4326_hybas_intersections\flood_DERP_RW_{rp_vis}_4326_{basin_id}.geojson'
flood_m = data_path / flood_plot_path
flood_gdf=gpd.read_file(flood_m)
layers_flood=[]
f_area_colors = {1:'blue', 3:'green'}
for f_area in flood_gdf.flood_area.unique():
    for f_depth in flood_gdf.depth_class.unique():
        subset_gdf = flood_gdf[(flood_gdf.depth_class==f_depth) & (flood_gdf.flood_area==f_area)]
        if not subset_gdf.empty:
            color_key=f'{f_area_colors[f_area]}_{f_depth}00'
            layers_flood.append(PolygonLayer.from_geopandas(subset_gdf, 
                                                            get_fill_color=miraca_colors[color_key], 
                                                            opacity=0.5, 
                                                            stroked=False))

layers=[]
if layer_basin is not None:
    layers.append(layer_basin)
else:
    print('No basin layer')

if layer_adapted_area is not None:
    layers.append(layer_adapted_area)
else:
    print('No protected area layer')

if layers_flood is not None:
    layers.extend(layers_flood)
else:
    print('No flood layers')

if layer_assets is not None:
    layers.extend(layers_assets)
else:
    print('No asset layer')
Voyager = 'https://basemaps.cartocdn.com/gl/voyager-gl-style/style.json'

if layer_od is not None:
    layers.append(layer_od)
else:
    print('No od layer')
m = Map(layers, show_tooltip=True, basemap_style=Voyager, view_state={"longitude": 7.91, "latitude": 49.91, "zoom": 11})


c:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\Lib\site-packages\lonboard\_geoarrow\ops\reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")
c:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\Lib\site-packages\lonboard\_geoarrow\ops\reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")
c:\Users\peregrin\AppData\Local\anaconda3\envs\ci_adapt_env\Lib\site-packages\lonboard\_geoarrow\ops\reproject.py:97: UserWarning: Input being reprojected to EPSG:4326 CRS
  warnings.warn("Input being reprojected to EPSG:4326 CRS")


DriverError: C:\Data\Floods\Germany\basin_intersections\DERP_RW_M_4326_hybas_intersections\flood_DERP_RW_M_4326_2080430320.geojson: No such file or directory

In [ ]:
m

In [ ]:
plot_assets=assets.drop(columns=['buffered', 'other_tags']).to_crs(epsg=4326)
plot_assets.head(3)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import cartopy.crs as ccrs
import cartopy.io.img_tiles as cimgt
import geopandas as gpd

# helper function
def zoomlevel_from_deg(deg): #https://stackoverflow.com/questions/30052990/how-to-use-openstreetmap-background-on-matplotlib-basemap
    "Calculate OSM zoom level from a span in degrees.  Adjust +/-1 as desired"
    from numpy import log2, clip, floor
    zoomlevel = int(clip(floor(log2(360) - log2(delta)),0,20 ))
    return zoomlevel 

# Convert basin to EPSG:4326 and get the total bounds
extent = basin.to_crs(epsg=4326).total_bounds

# Add clearance around the basin
clearance = 0.5  # 10% clearance around the basin bounds
lon_min, lat_min, lon_max, lat_max = extent
lon_range = lon_max - lon_min
lat_range = lat_max - lat_min

lon_min -= clearance * lon_range
lat_min -= clearance * lat_range
lon_max += clearance * lon_range
lat_max += clearance * lat_range

# Calculate aspect ratio
aspect_ratio = lon_range / lat_range

lon_i = lon_min + (lon_max - lon_min) / 2
lat_i = lat_min + (lat_max - lat_min) / 2
delta = 0.0016 # 0.0012 # 38 to 0.0002 degrees to avoid north pole
zoom = zoomlevel_from_deg(delta)-5 # 10 #  0-19 
print(f"Zoom Level: {zoom}")

# Create a plot with the OSM tiles
request_osm = cimgt.OSM()

fig = plt.figure(figsize=(10 * aspect_ratio, 10))
ax = plt.axes(projection=request_osm.crs)
ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())
ax.add_image(request_osm, zoom, alpha=0.5)  # Adjust zoom level as needed

# Plot the basin
basin.to_crs(epsg=4326).plot(ax=ax, facecolor='none', edgecolor='grey', linewidth=1, transform=ccrs.PlateCarree())

# Plot the assets
plot_assets.plot(ax=ax, color='grey', markersize=10, transform=ccrs.PlateCarree())

# Plot the flood areas
for f_area in flood_gdf.flood_area.unique():
    for f_depth in flood_gdf.depth_class.unique():
        subset_gdf = flood_gdf[(flood_gdf.depth_class==f_depth) & (flood_gdf.flood_area==f_area)]
        if not subset_gdf.empty:
            color_key=f'{f_area_colors[f_area]}_{f_depth}00'
            subset_gdf.to_crs(epsg=4326).plot(ax=ax, facecolor=miraca_colors[color_key], edgecolor=None, linewidth=2, alpha=0.5, transform=ccrs.PlateCarree())

# Plot the shortest paths
plot_assets_sps = assets_sps.drop(columns=['buffered', 'other_tags']).to_crs(epsg=4326)
plot_assets_sps.plot(ax=ax, color='black', markersize=10, transform=ccrs.PlateCarree())

# Plot the affected assets
plot_assets_affected = assets.iloc[affected_assets].drop(columns=['buffered', 'other_tags']).to_crs(epsg=4326)
plot_assets_affected.plot(ax=ax, color='red', markersize=10, transform=ccrs.PlateCarree())

# Plot the OD points
od_geoms_gdf.to_crs(epsg=4326).plot(ax=ax, color='blue', markersize=10, transform=ccrs.PlateCarree())

ax.gridlines(draw_labels=True)
plt.show()

In [ ]:
import cartopy
# Convert basin to EPSG:4326 and get the total bounds
extent_new = assets.to_crs(epsg=4326).total_bounds

# Add clearance around the basin
clearance = 0  # 10% clearance around the basin bounds

lon_min_new, lat_min_new, lon_max_new, lat_max_new = extent_new
lon_range_new = lon_max_new - lon_min_new
lat_range_new = lat_max_new - lat_min_new

lon_min_new -= clearance * lon_range_new
lat_min_new -= clearance * lat_range_new
lon_max_new += clearance * lon_range_new
lat_max_new += clearance * lat_range_new

# Calculate aspect ratio
aspect_ratio_new = lon_range_new / lat_range_new

lon_i_new = plot_assets.total_bounds[0] + (plot_assets.total_bounds[2] - plot_assets.total_bounds[0]) / 2
lat_i_new = plot_assets.total_bounds[1] + (plot_assets.total_bounds[3] - plot_assets.total_bounds[1]) / 2
delta_new = 0.0016 # 0.0012 # 38 to 0.0002 degrees to avoid north pole
zoom_new = zoomlevel_from_deg(delta_new)-5 # 10 #  0-19
print(f"Zoom Level: {zoom_new}")

fig = plt.figure(figsize=(10 * aspect_ratio_new, 10))
ax = plt.axes(projection=request_osm.crs)
ax.set_extent([lon_min_new, lon_max_new, lat_min_new, lat_max_new], crs=ccrs.PlateCarree())
ax.add_image(request_osm, zoom_new, alpha=0.5)  # Adjust zoom level as needed

# Plot the basin
basin.to_crs(epsg=4326).plot(ax=ax, facecolor='none', edgecolor='grey', linewidth=1, transform=ccrs.PlateCarree())

# Plot the assets
plot_assets.plot(ax=ax, color='grey', markersize=10, transform=ccrs.PlateCarree())

# Plot the flood areas
for f_area in flood_gdf.flood_area.unique():
    for f_depth in flood_gdf.depth_class.unique():
        subset_gdf = flood_gdf[(flood_gdf.depth_class==f_depth) & (flood_gdf.flood_area==f_area)]
        if not subset_gdf.empty:
            color_key=f'{f_area_colors[f_area]}_{f_depth}00'
            subset_gdf.to_crs(epsg=4326).plot(ax=ax, facecolor=miraca_colors[color_key], edgecolor=None, linewidth=2, alpha=0.5, transform=ccrs.PlateCarree())

# Plot the shortest paths
plot_assets_sps = assets_sps.drop(columns=['buffered', 'other_tags']).to_crs(epsg=4326)
plot_assets_sps.plot(ax=ax, color='black', markersize=10, transform=ccrs.PlateCarree())

# Plot the affected assets
plot_assets_affected = assets.iloc[affected_assets].drop(columns=['buffered', 'other_tags']).to_crs(epsg=4326)
plot_assets_affected.plot(ax=ax, color='red', markersize=10, transform=ccrs.PlateCarree())

# Plot the OD points
od_geoms_gdf.to_crs(epsg=4326).plot(ax=ax, color='cyan', markersize=10, transform=ccrs.PlateCarree())

ax.gridlines(draw_labels=True)

# add administrative boundaries
ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
plt.show()

In [ ]:
#!pip install cartopy
import cartopy.crs as ccrs
import cartopy
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pyproj

# Define the transformer to convert from EPSG:3857 to EPSG:4326
transformer = pyproj.Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

# Convert coordinates of o_geoms and d_geoms to EPSG:4326
o_geoms_4326 = [transformer.transform(geom.x, geom.y) for geom in o_geoms]
d_geoms_4326 = [transformer.transform(geom.x, geom.y) for geom in d_geoms]

# Create a map with the basin and the origins and destinations
fig = plt.figure(figsize=(20, 20))
ax = plt.axes(projection=ccrs.PlateCarree())

# Plot the basin (assuming basin is in EPSG:3857)
# If basin is in EPSG:3857, convert it to EPSG:4326
basin = basin.to_crs(epsg=4326)
basin.plot(ax=ax, facecolor='none', edgecolor='grey', linewidth=2, alpha = 0.5)

# Plot the assets
plot_assets.plot(ax=ax, facecolor='none', edgecolor='black', linewidth=2)

# Plot the flood areas
flood_gdf.plot(ax=ax, facecolor='none', edgecolor='blue', linewidth=1)

# Plot the protected area
# adapted_area.plot(ax=ax, facecolor='none', edgecolor='green', linewidth=2)

# Plot the shortest paths  
for (o,d), (path, demand) in shortest_paths.items():
    path_coords = [graph_v.nodes[node]['geometry'] for node in path]
    ax.plot([geom.x for geom in path_coords], [geom.y for geom in path_coords], 'b-', linewidth=2)

# Set the extent of the map based on basin boundaries
lon_min, lat_min, lon_max, lat_max = basin.total_bounds


# Plot the origins and destinations
for lon, lat in o_geoms_4326:
    ax.plot(lon, lat, 'ro', markersize=10)
for lon, lat in d_geoms_4326:
    ax.plot(lon, lat, 'ro', markersize=10)


# Calculate the range for longitude and latitude
lon_range = lon_max - lon_min
lat_range = lat_max - lat_min
# Add clearance around the basin
clearance = 0.6  # that is 60% of the range between min and max lon and lat on either side
# Apply clearance
lon_min -= clearance * lon_range
lat_min -= clearance * lat_range
lon_max += clearance * lon_range
lat_max += clearance * lat_range


ax.set_extent([lon_min, lon_max, lat_min, lat_max], crs=ccrs.PlateCarree())

# Add basemap
ax.add_feature(cartopy.feature.LAND, edgecolor='black')
ax.add_feature(cartopy.feature.BORDERS, linestyle=':')
ax.add_feature(cartopy.feature.LAKES, edgecolor='black')
ax.add_feature(cartopy.feature.RIVERS)    

# Add states and provinces
states_provinces = cartopy.feature.NaturalEarthFeature(
            category='cultural',  name='admin_1_states_provinces',
            scale='10m', facecolor='none')
ax.add_feature(states_provinces, edgecolor='black', zorder=10, linestyle = '-', linewidth=0.5)

# Add buildings
# buildings = cartopy.feature.NaturalEarthFeature(
#             category='cultural',  name='buildings',
#             scale='10m', facecolor='none')
# ax.add_feature(buildings, edgecolor='black', zorder=10, linestyle = '-', linewidth=0.5)

ax.gridlines(draw_labels=True)
plt.show()

In [ ]:
m

In [ ]:
adapted_edges=[(u,v,k) for u,v,k,attr in graph_v.edges(keys=True, data=True) if 'osm_id' in attr and 'l3_adaptation' in attr['osm_id']]
print(adapted_edges)
# shortest_paths
# assets_in_sps2=shortest_paths_assets[('node_164', 'node_10409')]
assets_in_sps2=shortest_paths_assets[('node_164', 'node_11238')]
# shortest_paths[('node_164', 'node_11238')]
# assets_in_sps2
# [(u,v) for (u,v), (path, demand) in shortest_paths_assets.items() if '111997044' in path]
disrupted_shortest_paths['flood_DERP_RW_L_4326_2080430320'][('node_164', 'node_11238')]==shortest_paths[('node_164', 'node_11238')]
print('shortest_paths: ', shortest_paths[('node_164', 'node_11238')])
print('disrupted_shortest_paths: ', disrupted_shortest_paths['flood_DERP_RW_L_4326_2080430320'][('node_164', 'node_11238')])





In [15]:

basins_path = gpd.read_file(data_path / r'Floods\basins\hybas_eu_lev01-12_v1c\hybas_eu_lev08_v1c_valid.shp')
regions_path = gpd.read_file(data_path / r'QGIS_data\rhineland_palatinate.geojson')

basin_list_tributaries, basin_list_full_flood = find_basin_lists(basins_path, regions_path)



In [ ]:
#!pip install seaborn
#PLOT INDIRECT DAMAGES AND DIRECT DAMGES FOR EACH HAZARD MAP SORTED BY SUMMED ADAPTATION COSTS
import matplotlib.pyplot as plt
import seaborn as sns
total_damages_adapted_df_mill=total_damages_adapted_df_mill.sort_values(by='summed_adaptation_costs')
fig, ax = plt.subplots(figsize=(10, 6))
sns.set_theme(style="whitegrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.barplot(x='summed_adaptation_costs', y='hazard_map', data=total_damages_adapted_df_mill, palette='viridis', ax=ax)
ax.set_xlabel('Summed adaptation costs [M€]')
ax.set_ylabel('Hazard map')
plt.show()

In [ ]:
# indirect damages adapted vs baseline
fig, ax = plt.subplots(figsize=(10, 6))
sns.set_theme(style="whitegrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.barplot(x='indirect damage adapted [M€]', y='hazard_map', data=total_damages_adapted_df_mill, palette='viridis', ax=ax)
ax.set_xlabel('Indirect damage adapted [M€]')
ax.set_ylabel('Hazard map')
plt.show()


In [ ]:
#!pip install seaborn
#adaptation cost for each basin/hazard map
import matplotlib.pyplot as plt
import seaborn as sns
total_damages_adapted_df_mill=total_damages_adapted_df_mill.sort_values(by='summed_adaptation_costs')
fig, ax = plt.subplots(figsize=(10, 6))
sns.set_theme(style="whitegrid")
sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
sns.barplot(x='summed_adaptation_costs', y='hazard_map', data=total_damages_adapted_df_mill, palette='viridis', ax=ax)
ax.set_xlabel('Summed adaptation costs [M€]')
ax.set_ylabel('Hazard map')
plt.show()